In [1]:
import scipy.io as scio
import numpy as np
import torch
import os
import pickle
import time
from Model import *
from itertools import chain
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.utils.data import DataLoader
from synthetic import simulate_lorenz_96, simulate_var
from utils import build_flags, time_split, save_result, evaluate_result, count_accuracy, loss_sparsity, loss_divergence, loss_mmd

In [2]:
parser = build_flags()
args = parser.parse_args(args=[])
args.seed = 2
args.num_nodes = 50
args.dims = 1
args.threshold = 0.5
args.time_length = 500
args.time_step = 10
args.epochs = 3000
args.batch_size = 128
args.lr = 1e-3
args.weight_decay = 1e-3
args.encoder_alpha = 0.02
args.decoder_alpha = 0.02
args.beta_sparsity = 0.05 #0.25   #log_sum  #1.25是F=40,N=30的最优值
args.beta_kl = 0.01        #JS散度
args.beta_mmd = 2      #MMD
args.encoder_hidden = 20
args.decoder_hidden = 20
args.encoder_dropout = 0.1
args.decoder_dropout = 0.2

In [3]:
sim2 = scio.loadmat('/home/omnisky/Public/ChenRongfa/sim4.mat')
X_np = sim2['ts']
GC = sim2['net']
GC = GC.mean(axis=0)
GC[GC != 0] = 1
GC[GC == 0] = 0
X_np_ori = X_np
X_np = X_np.transpose(1, 0)
X_np = X_np[:, :, np.newaxis]
X_np = np.array(time_split(X_np, step=10))
X_np = torch.FloatTensor(X_np)
data = X_np
data_loader = DataLoader(data, batch_size=128)

In [3]:
print(GC)

[[1. 1. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [4]:
print(X_np.shape)

torch.Size([9991, 50, 10, 1])


In [4]:
for idx in range(50):
    print('Begin training feature: {:04d}'.format(idx + 1))
    decoder_file = 'decoder' + str(idx) + '.pt'
    decoder_file = os.path.join('/home/jing_xuzijian/crf/Intrer_VAE_result/fmri_sim4', decoder_file)
    Inter_decoder = decoder(args.dims, args.decoder_hidden, args.time_step - 1, args.num_nodes, args.decoder_dropout, args.decoder_alpha)
    Inter_decoder = Inter_decoder.cuda()
    optimizer = optim.Adam(params = Inter_decoder.parameters(), lr=args.lr, weight_decay=args.weight_decay)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.5)
    loss_val = nn.MSELoss()
    best_loss = np.Inf
    for epoch in range(2000):
        scheduler.step()
        t = time.time()
        Loss = []
        mse_loss = []
        for batch_idx, data in enumerate(data_loader):
            data = data.cuda()
            target = data[:, idx, 1:, :]
            optimizer.zero_grad()
            inputs = data[:, :, :-1, :]
            pred = Inter_decoder(inputs, idx)
            mse = loss_val(pred, target)
            loss = mse

            loss.backward()
            optimizer.step()


            Loss.append(loss.item())
            mse_loss.append(mse.item())

        if epoch % 100 == 0:
            print('Feature: {:04d}'.format(idx + 1),
                'Epoch: {:04d}'.format(epoch),
                'Loss: {:.10f}'.format(np.mean(Loss)),
                'MSE_Loss: {:.10f}'.format(np.mean(mse_loss)),
                'time: {:.4f}s'.format(time.time() - t))

            
        if np.mean(mse_loss) < best_loss:
            best_loss = np.mean(mse_loss)
            torch.save(Inter_decoder.state_dict(), decoder_file)
            # print('Feature: {:04d}'.format(idx + 1),
            #       'Epoch: {:04d}'.format(epoch),
            #       'Loss: {:.10f}'.format(np.mean(Loss)),
            #       'mse_loss: {:.10f}'.format(np.mean(mse_loss)),
            #       'mmd_loss: {:.10f}'.format(np.mean(mmd_loss)),
            #       'time: {:.4f}s'.format(time.time() - t), file=log)

Begin training feature: 0001


/home/jing_xuzijian/crf/Intrer_VAE/Model.py:20: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(m.weight.data, gain=1.414)
/home/jing_xuzijian/anaconda3/envs/crf_base/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Feature: 0001 Epoch: 0000 Loss: 4.8551774357 MSE_Loss: 4.8551774357 time: 0.3910s
Feature: 0001 Epoch: 0100 Loss: 1.2090841502 MSE_Loss: 1.2090841502 time: 0.3378s
Feature: 0001 Epoch: 0200 Loss: 1.2123735826 MSE_Loss: 1.2123735826 time: 0.3387s
Feature: 0001 Epoch: 0300 Loss: 1.1832028982 MSE_Loss: 1.1832028982 time: 0.3402s
Feature: 0001 Epoch: 0400 Loss: 1.1763658591 MSE_Loss: 1.1763658591 time: 0.3404s
Feature: 0001 Epoch: 0500 Loss: 1.1784881127 MSE_Loss: 1.1784881127 time: 0.3399s
Feature: 0001 Epoch: 0600 Loss: 1.1752353899 MSE_Loss: 1.1752353899 time: 0.3396s
Feature: 0001 Epoch: 0700 Loss: 1.1801268617 MSE_Loss: 1.1801268617 time: 0.3392s
Feature: 0001 Epoch: 0800 Loss: 1.2245335752 MSE_Loss: 1.2245335752 time: 0.3389s
Feature: 0001 Epoch: 0900 Loss: 1.1747200776 MSE_Loss: 1.1747200776 time: 0.3390s
Feature: 0001 Epoch: 1000 Loss: 1.1659333200 MSE_Loss: 1.1659333200 time: 0.3385s
Feature: 0001 Epoch: 1100 Loss: 1.1504379816 MSE_Loss: 1.1504379816 time: 0.3385s
Feature: 0001 Ep

/home/jing_xuzijian/crf/Intrer_VAE/Model.py:20: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(m.weight.data, gain=1.414)


Feature: 0002 Epoch: 0000 Loss: 5.3167497780 MSE_Loss: 5.3167497780 time: 0.3391s
Feature: 0002 Epoch: 0100 Loss: 1.3171343781 MSE_Loss: 1.3171343781 time: 0.3386s
Feature: 0002 Epoch: 0200 Loss: 1.3128853591 MSE_Loss: 1.3128853591 time: 0.3386s
Feature: 0002 Epoch: 0300 Loss: 1.3410400680 MSE_Loss: 1.3410400680 time: 0.3385s
Feature: 0002 Epoch: 0400 Loss: 1.3208794609 MSE_Loss: 1.3208794609 time: 0.3383s
Feature: 0002 Epoch: 0500 Loss: 1.3279370462 MSE_Loss: 1.3279370462 time: 0.3386s
Feature: 0002 Epoch: 0600 Loss: 1.3424132564 MSE_Loss: 1.3424132564 time: 0.3385s
Feature: 0002 Epoch: 0700 Loss: 1.2972113649 MSE_Loss: 1.2972113649 time: 0.3386s
Feature: 0002 Epoch: 0800 Loss: 1.3070105627 MSE_Loss: 1.3070105627 time: 0.3387s
Feature: 0002 Epoch: 0900 Loss: 1.3059064889 MSE_Loss: 1.3059064889 time: 0.3389s
Feature: 0002 Epoch: 1000 Loss: 1.2738546125 MSE_Loss: 1.2738546125 time: 0.3391s
Feature: 0002 Epoch: 1100 Loss: 1.2825020680 MSE_Loss: 1.2825020680 time: 0.3392s
Feature: 0002 Ep

In [6]:
adj = []
for idx in range(50):
    decoder_file = 'decoder' + str(idx) + '.pt'
    decoder_file = os.path.join('/home/omnisky/Public/ChenRongfa/fmri_sim4', decoder_file)
    decoder_net = decoder(args.dims, args.decoder_hidden, args.time_step - 1, args.num_nodes, args.decoder_dropout, args.decoder_alpha)
    decoder_net.load_state_dict(torch.load(decoder_file))
    adj.append(decoder_net.adj[idx, :])

/home/omnisky/Public/ChenRongfa/Intrer_VAE/Model.py:20: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(m.weight.data, gain=1.414)


In [7]:
init_adj = torch.cat([temp.unsqueeze(0) for temp in adj], dim=0)
init_adj = init_adj.clone().detach()
print(init_adj.shape)

torch.Size([50, 50])


In [6]:
result, _ = evaluate_result(GC, init_adj.detach().numpy(), 0.5)
print(result)

{'accuracy': 0.9756, 'precision': 1.0, 'recall': 0.45045045045045046, 'F1': 0.6211180124223602, 'ROC_AUC': 0.9934911889704691, 'PR_AUC': 0.8701276465472852}


In [8]:
for idx in range(50):
    print('Begin training feature: {:04d}'.format(idx + 1))
    decoder_file = 'decoder' + str(idx) + '.pt'
    decoder_file = os.path.join('/home/jing_xuzijian/crf/Intrer_VAE_result/fmri_sim4', decoder_file)
    encoder_file = 'encoder' + str(idx) + '.pt'
    encoder_file = os.path.join('/home/jing_xuzijian/crf/Intrer_VAE_result/fmri_sim4', encoder_file)

    Inter_decoder = decoder(args.dims, args.decoder_hidden, args.time_step - 1, args.num_nodes, args.decoder_dropout, args.decoder_alpha)
    Inter_decoder.load_state_dict(torch.load(decoder_file))
    Inter_decoder = Inter_decoder.cuda()
    Inter_decoder.eval()

    Inter_encoder = encoder(init_adj, args.dims, args.encoder_hidden, args.dims, args.time_step - 1, args.encoder_dropout, args.encoder_alpha)
    Inter_encoder = Inter_encoder.cuda()

    optimizer = optim.Adam(Inter_encoder.parameters(), lr=args.lr, weight_decay=args.weight_decay)
    loss_mse = nn.MSELoss()
    best_loss = np.inf
    
    for epoch in range(2000):
        t = time.time()
        Loss = []
        MSE_loss = []
        SPA_loss = []
        KL_loss = []
        MMD_loss = []
        for batch_idx, data in enumerate(data_loader):
            optimizer.zero_grad()
            data = data.cuda()
            target = data[:, idx, 1:, :]
            inputs = data[:, :, :-1, :]

            mu, log_var = Inter_encoder(inputs)  #Inter_encoder(inputs, adj)
            sigma = torch.exp(log_var / 2)
            # sigma2 = torch.exp(log_var2 / 2)
            gamma = torch.randn(size = mu.size()).cuda()
            # theta = torch.randn(size = mu2.size()).cuda()
            gamma = mu + sigma * gamma
            # theta = mu2 + sigma2 * theta
            mask = torch.sigmoid(gamma) #* torch.sigmoid(theta) #* torch.sigmoid(theta + gamma)
            # gamma = torch.sigmoid(gamma)
            # theta = torch.sigmoid(theta)

            inputs = mask_inputs(mask, inputs)
            pred = Inter_decoder(inputs, idx)   #Inter_decoder(inputs, adj, idx)



            mse_loss = loss_mse(pred, target)
            spa_loss = loss_sparsity(mask, 'log_sum')
            kl_loss = loss_divergence(mask, 'JS')
            mmd_loss = loss_mmd(data[:, :, 1:, :], pred, idx)

            loss = mse_loss + args.beta_sparsity * spa_loss + args.beta_kl * kl_loss + args.beta_mmd * mmd_loss

            loss.backward()
            optimizer.step()

            Loss.append(loss.item())
            MSE_loss.append(mse_loss.item())
            SPA_loss.append(spa_loss.item())
            KL_loss.append(kl_loss.item())
            MMD_loss.append(mmd_loss.item())
        
        # if epoch == 500:
        #     optimizer.param_groups[0]['lr'] = args.lr/10

        if epoch % 100 == 0:
            print(  'Feature: {:04d}'.format(idx + 1),
                    'Epoch: {:04d}'.format(epoch),
                    'Loss: {:.10f}'.format(np.mean(Loss)),
                    'MSE_Loss: {:.10f}'.format(np.mean(MSE_loss)),
                    'Sparsity_loss: {:.10f}'.format(np.mean(SPA_loss)),
                    'KL_loss: {:.10f}'.format(np.mean(KL_loss)),
                    'MMD_loss: {:.10f}'.format(np.mean(MMD_loss)),
                    'time: {:.4f}s'.format(time.time() - t))
                
        if np.mean(Loss) < best_loss:
            best_loss = np.mean(Loss)
            #M[idx, :] = 
            # gamma_matrix[idx, :] = gamma.squeeze().mean(dim=2).mean(dim=1)
            # theta_matrix[idx, :] = theta.squeeze().mean(dim=2).mean(dim=1)
            # torch.save({
            #             'encoder_state_dict': Inter_encoder.state_dict(),
            #             'decoder_state_dict': Inter_decoder.state_dict(),
            #                 # 'adj' : adj

            #             }, encoder_file)
            torch.save(Inter_encoder.state_dict(), encoder_file)
                # np.save(save_file + str(idx) + '.npy', mask.cpu().detach().numpy())

            # print('Feature: {:04d}'.format(idx + 1),
            #       'Epoch: {:04d}'.format(epoch),
            #       'Loss: {:.10f}'.format(np.mean(Loss)),
            #       'mse_loss: {:.10f}'.format(np.mean(MSE_loss)),
            #       'Sparsity_loss: {:.10f}'.format(np.mean(SPA_loss)),
            #       'KL_loss: {:.10f}'.format(np.mean(KL_loss)),
            #       #'mmd_loss: {:.10f}'.format(np.mean(mmd_loss)),
            #       # 'time: {:.4f}s'.format(time.time() - t), file=log)

Begin training feature: 0001


/home/jing_xuzijian/crf/Intrer_VAE/Model.py:20: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(m.weight.data, gain=1.414)


Feature: 0001 Epoch: 0000 Loss: 1.8476178299 MSE_Loss: 1.3537910249 Sparsity_loss: 8.2673076919 KL_loss: 0.0762382840 MMD_loss: 0.0398495171 time: 0.8757s
Feature: 0001 Epoch: 0100 Loss: 1.2310018343 MSE_Loss: 1.1195286663 Sparsity_loss: 0.7552484304 KL_loss: 0.0488326616 MMD_loss: 0.0366112141 time: 0.8209s
Feature: 0001 Epoch: 0200 Loss: 1.2309947301 MSE_Loss: 1.1195175399 Sparsity_loss: 0.7553394475 KL_loss: 0.0488469109 MMD_loss: 0.0366108716 time: 0.8232s
Feature: 0001 Epoch: 0300 Loss: 1.2309896584 MSE_Loss: 1.1195117839 Sparsity_loss: 0.7553491215 KL_loss: 0.0488573655 MMD_loss: 0.0366109157 time: 0.8209s
Feature: 0001 Epoch: 0400 Loss: 1.2309890382 MSE_Loss: 1.1195665387 Sparsity_loss: 0.7542707573 KL_loss: 0.0486605840 MMD_loss: 0.0366111749 time: 0.8247s
Feature: 0001 Epoch: 0500 Loss: 1.2309903510 MSE_Loss: 1.1195101512 Sparsity_loss: 0.7553975069 KL_loss: 0.0488720748 MMD_loss: 0.0366108014 time: 0.8216s
Feature: 0001 Epoch: 0600 Loss: 1.2309904559 MSE_Loss: 1.1195167394 Sp

In [8]:
causality_matrix = []
total_gamma_matrix = []
total_theta_matrix = []
#init_adj = torch.eye(20)
for idx in range(2):
    encoder_file = 'encoder' + str(idx) + '.pt'
    encoder_file = os.path.join('/home/omnisky/Public/ChenRongfa/fmri_sim4', encoder_file)
    est_net = encoder(init_adj, args.dims, args.encoder_hidden, args.dims, args.time_step - 1, args.encoder_dropout, args.encoder_alpha)
    est_net.load_state_dict(torch.load(encoder_file))
    # est_net = est_net.to('cuda:1')
    est_net.eval()
    inputs = X_np[:, :, :-1, :]#.to('cuda:1')   #:-1和1:有什么区别
    mu, log_var = est_net(inputs)
    # mu = mu.cpu().detach()
    # log_var = log_var.cpu().detach()
    sigma = torch.exp(log_var / 2)
    # sigma2 = torch.exp(log_var2 / 2)
    gamma = torch.randn(size = mu.size())
    # theta = torch.randn(size = mu1.size())
    gamma = mu + sigma * gamma
    # theta = mu2 + sigma2* theta
    mask_matrix = torch.sigmoid(gamma) #* torch.sigmoid(theta)
    mask_matrix = mask_matrix.squeeze()
    causality_matrix.append(mask_matrix)
    # gamma_matrix = torch.sigmoid(gamma)
    # gamma_matrix = gamma_matrix.squeeze()
    # total_gamma_matrix.append(gamma_matrix)
    # theta_matrix = torch.sigmoid(theta)
    # theta_matrix = theta_matrix.squeeze()
    # total_theta_matrix.append(theta_matrix)

# np.save('/home/omnisky/Public/ChenRongfa/Intrer_VAE_result/result_fmri_concat/causality_matrix_10',causality_matrix)

causality_matrix = torch.stack(causality_matrix, dim=1)

# total_gamma_matrix = torch.stack(total_gamma_matrix, dim=1)
# total_theta_matrix = torch.stack(total_theta_matrix, dim=1)

In [8]:
adj_gca = causality_matrix.mean(dim=3).mean(dim=0)
np.save('/home/omnisky/Public/ChenRongfa/Intrer_VAE_result/result_fmri_concat/adj_gca_10',adj_gca.detach().numpy())
# gamma_adj = total_gamma_matrix.mean(dim=3).mean(dim=0)
# theta_adj = total_theta_matrix.mean(dim=3).mean(dim=0)
print(adj_gca.shape)

torch.Size([5, 50])


In [4]:
adj_gca = np.load('/home/omnisky/Public/ChenRongfa/Intrer_VAE_result/result_fmri_concat/adj_gca_1.npy')
for idx in range(2,11):
    file = '/home/omnisky/Public/ChenRongfa/Intrer_VAE_result/result_fmri_concat/adj_gca_' + str(idx) + '.npy'
    adj = np.load(file)
    adj_gca = np.concatenate((adj_gca, adj))

print(adj_gca.shape)

(50, 50)


In [11]:
result, _ = evaluate_result(GC, adj_gca, 0.5)
print(result)

{'accuracy': 0.9332, 'precision': 0.39928057553956836, 'recall': 1.0, 'F1': 0.5706940874035991, 'ROC_AUC': 0.9834696563453366, 'PR_AUC': 0.6693590152422391}


In [7]:
print(adj_gca[0,:])

[9.9999756e-01 9.9985868e-01 9.7211050e-06 9.2710952e-06 9.9985385e-01
 9.3774725e-06 9.3284125e-06 9.7708935e-06 9.5543246e-06 8.9184632e-06
 9.5961177e-06 9.3959152e-06 9.6411231e-06 8.6593700e-06 9.5046353e-06
 8.5324618e-06 9.4927373e-06 8.8207071e-06 9.2803066e-06 9.4160778e-06
 9.1965385e-06 9.1515576e-06 9.2759246e-06 9.6836839e-06 9.8320315e-06
 9.2732207e-06 9.3022054e-06 9.2657292e-06 9.5832602e-06 9.3807803e-06
 9.1514676e-06 1.0003901e-05 8.9867190e-06 9.2027703e-06 9.3525477e-06
 9.2896389e-06 9.1430720e-06 8.7794806e-06 9.5598398e-06 8.9996784e-06
 8.7186336e-06 8.8284305e-06 9.1528018e-06 9.0607837e-06 9.3686122e-06
 9.4386169e-06 9.4497600e-06 8.9849527e-06 9.0072836e-06 9.6226840e-06]


In [6]:
print(GC[0,:])

[1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]


In [13]:
from contrast_models.clstm import cLSTM, train_model_ista
X = torch.FloatTensor(X_np_ori[np.newaxis])
X = X.cuda()
clstm = cLSTM(X.shape[-1], hidden=100).cuda()
train_loss_list = train_model_ista(
    clstm, X, context=10, lam=0.30, lam_ridge=1e-2, lr=1e-3, max_iter=4000,
    check_every=1000)
adj_clstm = clstm.GC(threshold=False).cpu().data.numpy()
result_clstm, _ = evaluate_result(GC, adj_clstm, 0)

----------Iter = 1000----------
Loss = 8.797492
Variable usage = 100.00%
----------Iter = 2000----------
Loss = 7.815337
Variable usage = 100.00%
----------Iter = 3000----------
Loss = 6.890838
Variable usage = 100.00%
----------Iter = 4000----------
Loss = 6.137646
Variable usage = 40.00%


In [20]:
result_clstm, _ = evaluate_result(GC, adj_clstm, 0)
print(result_clstm)

{'accuracy': 0.79, 'precision': 0.5, 'recall': 0.9523809523809523, 'F1': 0.6557377049180327, 'ROC_AUC': 0.9171187462326702, 'PR_AUC': 0.7998405051892206}


In [11]:
print(init_adj)

tensor([[ 0.0000,  0.1223,  0.0164, -0.0233,  0.1127, -0.0018, -0.0035,  0.0089,
         -0.0077, -0.0020],
        [ 0.1087,  0.0000,  0.1149,  0.0153,  0.0224,  0.0024, -0.0021,  0.0169,
          0.0092,  0.0034],
        [ 0.0107,  0.0833,  0.0000,  0.0898,  0.0205,  0.0006, -0.0168,  0.0802,
          0.0171,  0.0068],
        [-0.0295,  0.0133,  0.1332,  0.0000,  0.1441, -0.0039, -0.0055,  0.0334,
          0.0107,  0.0026],
        [ 0.1025,  0.0163,  0.0291,  0.1180,  0.0000, -0.0009,  0.0013,  0.0130,
          0.0054,  0.0020],
        [ 0.0033, -0.0040, -0.0006, -0.0124, -0.0041,  0.0000,  0.1226,  0.0161,
         -0.0154,  0.1225],
        [-0.0032, -0.0062, -0.0191, -0.0062,  0.0035,  0.1099,  0.0000,  0.1111,
          0.0191,  0.0170],
        [ 0.0071,  0.0206,  0.0835,  0.0219,  0.0126,  0.0144,  0.0867,  0.0000,
          0.0969,  0.0197],
        [-0.0039,  0.0053,  0.0193,  0.0080,  0.0010, -0.0139,  0.0170,  0.1131,
          0.0000,  0.1099],
        [-0.0013,  

In [15]:
new_value = 1
for i in range(len(GC)):
    for j in range(i+1):
        if i == j or j == len(GC[0])-1:
            GC[i][j] = new_value

In [16]:
print(GC)

[[1. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [15]:
data = scio.loadmat('/home/jing_xuzijian/crf/fmri模拟/sims/sim2.mat')

In [18]:
GC = data['net']
shuju = data['ts']
cc = GC.mean(axis=0)
print(shuju.shape)

(10000, 10)


In [13]:
cc[cc>0] = 1
cc[cc<=0] = 0
print(cc)

[[0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
